In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division
from __future__ import print_function

import sys, os, gc, math
import numpy as np
from scipy.fftpack import dct,idct
from keras.datasets import mnist, fashion_mnist
from PIL import Image


sys.path.append('../')

from models.util import *


#Seed used for choosing classes, training points, and test points.
#SEED = 14
SEED=11

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
num_samples = 500
sqrt_n = 125
input_shape=(sqrt_n,sqrt_n,1)
n = sqrt_n*sqrt_n
k = 17

In [3]:
#Load MNIST data
(X_train, _), (X_test, _) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

m_data_small = np.concatenate((X_train,X_test))
            
            
m_data = np.zeros((m_data_small.shape[0],sqrt_n,sqrt_n,1))
            
for i in range(m_data.shape[0]):
    img = m_data_small[i,:,:,0]
    img = Image.fromarray(img)
    basewidth = sqrt_n
    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize), Image.ANTIALIAS)
    m_data[i,:,:,0] = np.asarray(img)
                

In [4]:
#Normalize the data
m_data = m_data/255.0

In [5]:
#Check MNIST results for 1000 random images
#n=40000, k=40, t <= (n/27*k) ~ 37.03
c=2.0
t = int(n/27/k/c)
subset_idx = np.random.choice(np.arange(m_data.shape[0]),num_samples)
m_data_sub = m_data[subset_idx]
m_data_y = np.zeros((num_samples,sqrt_n,sqrt_n))
t_values = np.zeros(num_samples)
for i in range(num_samples):
    #first sample an element from the data
    x = m_data_sub[i,:,:,0].flatten()
    #Now sample a t - must be atleast 1
    t_l = np.random.randint(1,t)
    t_values[i] = t_l
    #Now samnple the an index set from [n] with cardinality = t_l
    s = np.random.choice(np.arange(n),t_l)
    e = np.zeros(n)
    #Now create the vector e
    #pick a value for each element between 0 and 1 as the images are normalized
    for j in range(t_l):
        e[s[j]] = np.random.uniform()
    y = x + e
    m_data_y[i,:,:] = y.reshape((sqrt_n,sqrt_n))        

In [6]:
%%capture one
#MNIST IHT
errors_l2 = np.zeros(m_data_y.shape[0])
errors_inf = np.zeros(m_data_y.shape[0])
bot_l2 = np.zeros(m_data_y.shape[0])
tau = np.zeros(m_data_y.shape[0])
diff_l2 = np.zeros(m_data_y.shape[0])
diff_inf = np.zeros(m_data_y.shape[0])
T_values = np.zeros(m_data_y.shape[0])

for i in range(num_samples):
    y = m_data_y[i,:,:].flatten()
    x = m_data_sub[i,:,:].flatten()
    
    #Get actual top k and bottom k
    x_hat_top_k, x_hat_bot_k =  get_top_bot_k_vec(dct(x, norm='ortho'),k=k)
    #Now we need to calculate T
    e = y - x 
    #Get the constant that we need to set to zero as a function of T
    a = np.sqrt(np.linalg.norm(x_hat_top_k)**2 + np.linalg.norm(e)**2)
    #Lets get rho
    rho = np.sqrt(27)*np.sqrt(c*k*t_values[i]/float(n))
    
    #We want to find a T such that rho^T a is very small - we use 1e-20 
    T = int((np.log(1e-20) - np.log(a))/np.log(rho))
    T_values[i] = T
    #Get top k approx
    x_hat_approx,e_hat_approx = iht(y,int(t_values[i]),T=T,k=k)
    
    #Note the errors
    errors_l2[i] = np.linalg.norm(x_hat_top_k.flatten()- x_hat_approx.flatten())
    errors_inf[i] = np.linalg.norm(x_hat_top_k.flatten()- x_hat_approx.flatten(), ord=np.inf)

    #Note the norm of bottom k coefficients
    bot_l2[i] = np.linalg.norm(x_hat_bot_k)

    #Get the multiplicative constant
    c_l2 = np.sqrt(4*c*k*t_values[i]/n)
    c_inf = np.sqrt(2*c*t_values[i]/n)
    #Calculate tau
    tau[i] = (np.sqrt(3)*np.sqrt(1 + 2*np.sqrt(c*k*t_values[i]/n)))/(1-rho)
    
    #Calculate the difference from the upper bound
    diff_l2[i] = (c_l2*tau[i]*bot_l2[i]) - errors_l2[i] 
    diff_inf[i] = (c_inf*tau[i]*bot_l2[i]) - errors_inf[i]

In [7]:
print(np.mean(t_values), 
      np.mean(tau), 
      np.mean(errors_l2), 
      np.mean(diff_l2), 
      np.mean(errors_inf), 
      np.mean(diff_inf))

8.486 12.880381823912398 0.6865346126117067 128.30272947652045 0.44548317008755417 21.675993119772038


In [8]:
mnist_tup = (m_data_y, m_data_sub, t_values,tau,errors_l2, errors_inf, diff_l2, diff_inf )

import pickle
with open('data/mnist_tuple_theory.pickle', 'wb') as f:
    pickle.dump(mnist_tup, f)